# MedPC event reading example

Demonstration of loading a set of subjects data from a MedPC format and plotting by subject and event.

In [ ]:
import glob
import numpy as np
import pandas as pd
import behapy.medpc as medpc
import seaborn as sns

Map event indices to event names.

In [ ]:
EVENTS = {1: 'Event 1',
          2: 'Event 2',
          3: 'Event 3',
          4: 'Event 4',
          5: 'Event 5',
          6: 'Event 6'}

Load data from all files prefixed with 'Backup of Box...' into a single DataFrame.

In [ ]:
all_info = []
all_events = []
for fn in glob.glob('Backup of Box*.txt'):
    variables = medpc.parse_file(fn)
    info = medpc.experiment_info(variables)
    events = medpc.get_events(variables['Z'], variables['Y'], EVENTS)
    events['subject'] = info['subject']
    events.set_index(['subject', 'timestamp'], inplace=True)
    all_info.append(info)
    all_events.append(events)

if all_info:
    info_df = pd.DataFrame(all_info)
    info_df.set_index('subject', inplace=True)
if all_events:
    events_df = pd.concat(all_events)

Bin events minute-by-minute for each subject and event.

In [ ]:
window = pd.Timedelta(60, 's')
events_df = events_df.reset_index()
binedges = np.arange(pd.Timedelta(0),
                     events_df['timestamp'].max() + window,
                     window)
bins = pd.cut(events_df['timestamp'], bins=binedges,
              labels=(binedges[:-1] + np.diff(binedges) / 2).astype('timedelta64[s]').astype(int))
counts_df = events_df.groupby(['subject', 'event', bins]).count()
counts_df = counts_df.rename(columns={'timestamp': 'count'})
counts_df

Plot each event.

In [ ]:
sns.set_theme()
sns.relplot(data=counts_df, x='Timestamp', y='Count', hue='Subject',
            row='Event', kind='line', height=5, aspect=3,
            facet_kws=dict(sharex=True, sharey=False))